In [89]:
import os
import csv
import numpy as np
from glob import glob
import json

In [17]:
print(os.listdir('../data'))
path_base = '../data'
path_to_annot = os.path.join(path_base, 'anno')

print(path_to_annot)

['anno', 'val2017', 'train2017']
../data/anno


In [22]:
train_annot = glob(os.path.join(path_to_annot, 'train*/*.json'), recursive=True)
val_annot = glob(os.path.join(path_to_annot, 'val*/*.json'), recursive=True)

In [23]:
print('train annot size: ', len(train_annot))
print('val annot size: ', len(val_annot))

train annot size:  10000
val annot size:  4167


In [31]:
training_img = glob(os.path.join(path_base, 'train*/*'))
print('train imgs size: ', len(training_img))
val_img = glob(os.path.join(path_base, 'val*/*'))
print('val imgs size: ', len(val_img))

train imgs size:  10000
val imgs size:  4167


In [42]:
def read_json(path_to_file):
    with open(path_to_file) as file:
        json_data = json.load(file)
    return json_data

In [50]:
classes = set()
i = 0
for f in train_annot + val_annot:
    j_data = read_json(f) 
    i += 1
    for label in j_data['labels']:
        classes.add(label)
print('total number of labels',  len(classes))

total number of labels 68


In [77]:
labels_to_find = ['nudity', 'face', 'credit_card', 'passport', 'ticket', 'disability_physical', 
                  'fingerprint', 'safe']
for label in classes:
    for l in labels_to_find:
        if l in label:
            print(label)

a13_full_nudity
a10_face_partial
a39_disability_physical
a7_fingerprint
a38_ticket
a31_passport
a0_safe
a9_face_complete
a12_semi_nudity
a30_credit_card


In [87]:
def get_output_dict(list_to_process, labels_to_predict):
    output_data = {}
    for f in list_to_process:
        j_data = read_json(f)
        values = np.zeros(len(labels_to_predict))
        for label in j_data['labels']:
            for idx, l in enumerate(labels_to_predict):
                if l in label:
                    values[idx] = 1
        img_path = j_data['image_path'].split('/')[-1]
        output_data[img_path] = values
    return output_data
                    

In [100]:
step = 200
start = 0
data = val_annot
data_name = 'val'
for end in range(step, len(data), step):
    if end + step >= len(data):
        end += len(data) - end
    result = get_output_dict(data[start:end], labels_to_find)
    with open('{}_{}_{}.csv'.format(data_name,start, end), 'w') as f:
        writer = csv.writer(f)
        for key in result.keys():
            writer.writerow((key, result[key]))
    start = end

[1]
